In [1]:
#!/usr/bin/env python3.7
# -*- coding: utf-8 -*-
# coding: utf-8

In [2]:
from pprint import pprint, pformat
from typing import List, Tuple
import tensorflow as tf
from os import environ 
from os import path
import json
import sys

In [3]:
import importlib

In [4]:
if "util" in sys.modules:
    importlib.reload(util)
else:
    import util
Actions     = util.keys.Actions
Keys        = util.keys.Keys
Models      = util.keys.Models
Runs        = util.keys.Runs
lclean      = util.functions.lclean
ensure_path = util.functions.ensure_path

In [5]:
from keras.engine.functional import Functional as KFunctional

In [6]:
from keras.utils.vis_utils import plot_model
from IPython import display

In [7]:
environ["TF_KERAS"] = "True"
import keras_gpt_2 # load_trained_model_from_checkpoint, get_bpe_from_files, generate

In [8]:
import tensorflow_model_optimization as tfmot

In [9]:
outlist, model = [], False

In [10]:
TARGET = Actions.GENERATE

In [11]:
config = {
    "run_name":          Runs.bible_en_plus,
    "checkpoint":        'model-4250',
    "length":            1000,
    "temperature":       .70,
    "k_top":             20,
    "batch_size":        2,
}

In [12]:
prompt: List[str] = [Keys.START_KEY] * config["batch_size"]

In [13]:
stuff_path = path.join('checkpoint', config["run_name"])

In [14]:
model_params = {
    "model_folder":    stuff_path,
    "config_path":     path.join(stuff_path, 'hparams.json'),
    "checkpoint_path": path.join(stuff_path, config["checkpoint"]), # may change
    "encoder_path":    path.join(stuff_path, 'encoder.json'),
    "vocab_path":      path.join(stuff_path, 'vocab.bpe'),
}

In [15]:
if not(model):
    model = keras_gpt_2.load_trained_model_from_checkpoint(model_params["config_path"], model_params["checkpoint_path"])
    bpe   = keras_gpt_2.get_bpe_from_files(model_params["encoder_path"], model_params["vocab_path"])

In [16]:
if (TARGET == Actions.SAVE):
    target = path.join(ensure_path(stuff_path, "artifacts"), config["checkpoint"])
    model.save(filepath=target+".h5", overwrite=True, include_optimizer=True)
    with open(path.join(target+".tflite"), "wb") as tfilete:
        tfilete.write(tf.lite.TFLiteConverter.from_keras_model(model).convert())

    target = path.join(ensure_path(stuff_path, "artifacts"), config["checkpoint"])
    model.save(filepath=target+".h5", overwrite=True, include_optimizer=True)
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.target_spec.supported_ops = [tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8, tf.lite.OpsSet.TFLITE_BUILTINS]
    with open(path.join(target+".i16wi8.tflite"), "wb") as tfilete:
        tfilete.write(converter.convert())

In [17]:
if (TARGET == Actions.GRAPH_TXT):
    model.summary()

In [18]:
if (TARGET == Actions.GRAPH):
    display.display(plot_model(model,
        to_file=path.join("renders", config["run_name"] + '.png'),
        show_shapes=True, show_dtype=False, show_layer_names=True,
        rankdir='TB', expand_nested=False, dpi=96, layer_range=None
    ))

In [ ]:
if (TARGET == Actions.GENERATE):
    outlist += (keras_gpt_2.generate(model=model, bpe=bpe, texts=(prompt if prompt else None), length=config["length"], top_k=config["k_top"], temperature=config["temperature"]))

In [ ]:
pprint(list(lclean(outlist)))

In [ ]:
#pprint(json.dumps(outlist, sort_keys=True, indent=4))
for l in outlist:
    print(" ~ ".center(100, "-"))
    print(l)